In [1]:
import requests
page = requests.get("https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p1.html")
page

<Response [200]>

In [2]:
from bs4 import BeautifulSoup as bs

soup = bs(page.content, 'html.parser')

#branch = soup.find_all(class_='ProductList-content')
#branch = soup.select("div.ProductList-content li")
#item = branch[0].find('li')
#js = item.find('script')


In [3]:
product = soup.find('article', class_='ProductDisplay')

ProductName = product.find('div', class_='ProductDisplay-name')
product_name_a = ProductName.find('a')

print(product_name_a['title'])
print("https://wine.com.br" + product_name_a['href'])

country = product.find(class_="Country")
print(country.span.text)

wine_type = product.find(class_="ProductDisplay-type")
print(wine_type.span.text)

wine_description = product.find(class_="ProductDisplay-descriptionText")
print(wine_description.text)
      
# print(product_name_a.prettify())

Pérez Cruz Limited Edition D.O. Valle del Maipo Cabernet Franc 2016
https://wine.com.br/vinhos/perez-cruz-limited-edition-d-o-valle-del-maipo-cabernet-franc-2016/prod20081.html
Chile
tinto
Frutado, taninos presentes, médio corpo, fresco, amadeirado, especiarias


In [80]:
# Buscando Notas e numero de avaliações
import re

wine_rating = product.find('div',class_='Rating')
wine_evaluation_tag = wine_rating.find('evaluation-tag')
if (wine_evaluation_tag):
    wine_evaluation = re.compile(r"\d.\d").search(wine_evaluation_tag.prettify()).group()
    wine_rating_count = re.search(r"\d+", wine_rating.find(class_='Rating-count').text).group()
else:
    wine_evaluation = None
    wine_rating_count = 0

print(str(wine_rating))
print(wine_evaluation)
print(wine_rating_count)

<div class="Rating">
<div class="Rating-stars">
<div class="vue-evaluation-tag">
<evaluation-tag :evaluation="4.0"></evaluation-tag>
</div>
</div>
<a class="Rating-count" href="/vinhos/perez-cruz-limited-edition-d-o-valle-del-maipo-cabernet-franc-2016/prod20081.html#Comments">(7)</a>
</div>
4.0
7


In [83]:
price_box = product.find(class_="ProductDisplay-priceBox")

lowest_price = price_box.find(class_="Price--lowestPrice").find(class_="Price-raw").text
full_price = price_box.find(class_="Price--fullPrice").find(class_="Price-raw").text
print(lowest_price)
print(full_price)

discount = 1-float(lowest_price)/float(full_price)
print(discount)

145.76
180.90
0.1942509673852958


In [89]:
pages_list = soup.find('div', class_='Pagination').ul.find_all('li')
int(pages_list[len(pages_list)-1].a.text)

44

In [82]:
wine_list = soup.find('div', class_='ProductList-content').ul.find_all('li')

for wine_item in wine_list:
    wine = wine_item.find('article', class_='ProductDisplay')

    wine_name = wine.find('div', class_='ProductDisplay-name').a['title']
    wine_link = "https://wine.com.br" + wine.find('div', class_='ProductDisplay-name').a['href']
    wine_country = wine.find(class_="Country").span.text
    wine_type = wine.find(class_="ProductDisplay-type").span.text
    wine_description = wine.find(class_="ProductDisplay-descriptionText").text
    
    wine_rating = wine.find('div',class_='Rating')
    wine_evaluation_tag = wine_rating.find('evaluation-tag')
    if (wine_evaluation_tag):
        wine_evaluation = re.compile(r"\d.\d").search(wine_evaluation_tag.prettify()).group()
        wine_rating_count = re.search(r"\d+", wine_rating.find(class_='Rating-count').text).group()
    else:
        wine_evaluation = None
        wine_rating_count = 0
    
    price_box = wine.find(class_="ProductDisplay-priceBox")

    lowest_price = price_box.find(class_="Price--lowestPrice").find(class_="Price-raw").text
    full_price = price_box.find(class_="Price--fullPrice").find(class_="Price-raw").text
    discount = 1-float(lowest_price)/float(full_price)
    
    
    print(wine_name,";", wine_evaluation, ";", wine_rating_count, ";", wine_country, ";", wine_type, ";", lowest_price, ";", full_price, ";", discount,";", wine_description, ";", wine_link)

Pérez Cruz Limited Edition D.O. Valle del Maipo Cabernet Franc 2016 ; 4.0 ; 7 ; Chile ; tinto ; 145.76 ; 180.90 ; 0.1942509673852958 ; Frutado, taninos presentes, médio corpo, fresco, amadeirado, especiarias ; https://wine.com.br/vinhos/perez-cruz-limited-edition-d-o-valle-del-maipo-cabernet-franc-2016/prod20081.html
Finca La Chamiza Martín Alsina Malbec 2014 ; None ; 0 ; Argentina ; tinto ; 292.82 ; 495.90 ; 0.4095180479935471 ; Frutado, amadeirado, corpo de médio para encorpado, taninos presentes, acidez agradável ; https://wine.com.br/vinhos/finca-la-chamiza-martin-alsina-malbec-2014/prod20028.html
Domaine L'Ostal Cazes Grand Vin 2015 ; None ; 0 ; França ; tinto ; 258.71 ; 335.90 ; 0.22980053587377192 ; Encorpado, boa acidez, taninos macios ; https://wine.com.br/vinhos/domaine-l-ostal-cazes-grand-vin-2015/prod20143.html
Maycas del Limarí Reserva Especial Pinot Noir 2017 ; 4.5 ; 23 ; Chile ; tinto ; 166.94 ; 206.90 ; 0.19313678105364918 ; Bom corpo, acidez presente, taninos sedosos, 

In [146]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import time

URL_RAIZ="https://www.wine.com.br"
URL_BASE = URL_RAIZ+"/vinhos/tinto/cVINHOS-atTIPO_TINTO-p{0}.html"


HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36')
}



In [148]:
def get_num_pages(soup):
    
    pages_list = soup.find('div', class_='Pagination').ul.find_all('li')
    num_pages = int(pages_list[len(pages_list)-1].a.text) + 1
    
    return num_pages

def scrape_soup(soup):
    
    list_name=[]
    list_link, list_country, list_type, list_description= [], [], [], []
    list_evaluation, list_rating_count= [], []
    list_lowest_price, list_full_price, list_discount= [], [], []
    
    wine_list = soup.find('div', class_='ProductList-content').ul.find_all('li')

    for wine_item in wine_list:
        
        try:
            wine = wine_item.find('article', class_='ProductDisplay')

            wine_name = wine.find('div', class_='ProductDisplay-name').a['title']
            wine_link = "https://wine.com.br" + wine.find('div', class_='ProductDisplay-name').a['href']
            wine_country = wine.find(class_="Country").span.text
            wine_type = wine.find(class_="ProductDisplay-type").span.text
            wine_description = wine.find(class_="ProductDisplay-descriptionText").text
        
            wine_rating = wine.find('div',class_='Rating')
            wine_evaluation_tag = wine_rating.find('evaluation-tag')
            if (wine_evaluation_tag):
                wine_evaluation = float(re.compile(r"\d.\d").search(wine_evaluation_tag.prettify()).group())
                wine_rating_count = int(re.search(r"\d+", wine_rating.find(class_='Rating-count').text).group())
            else:
                wine_evaluation = 0
                wine_rating_count = 0
    
        
            price_box = wine.find(class_="ProductDisplay-priceBox")

            lowest_price = float(price_box.find(class_="Price--lowestPrice").find(class_="Price-raw").text)
            full_price = float(price_box.find(class_="Price--fullPrice").find(class_="Price-raw").text)
            discount = 1-float(lowest_price)/float(full_price)
        except:
            print('skip incomplete item')
            continue
        
        list_name.append(wine_name)
        list_link.append(wine_link)
        list_country.append(wine_country)
        list_type.append(wine_type)
        list_description.append(wine_description)
        
        list_evaluation.append(wine_evaluation)
        list_rating_count.append(wine_rating_count)

        list_lowest_price.append(lowest_price)
        list_full_price.append(full_price)
        list_discount.append(discount)
        
        ###print(wine_name,";", wine_evaluation, ";", wine_rating_count, ";", wine_country, ";", wine_type, ";", lowest_price, ";", full_price, ";", discount,";", wine_link)
    
    df = pd.DataFrame({
        'wine_name': list_name,
        'link': list_link,
        'country': list_country,
        'type': list_type,
        'description': list_description,
        'evaluation': list_evaluation,
        'rating_count': list_rating_count,
        'lowest_price': list_lowest_price,
        'full_price': list_full_price,
        'discount': list_discount
    })
    
    return df
        
    

def get_soup(page_url, retry_count=0):
    
    session= requests.Session()
    
    try:
        response = session.get(page_url, headers=HEADERS)
    except:
        retry_count += 1
        if retry_count <= 3:
            session = requests.Session()
            get_soup(page_url, retry_count)
        else:
            raise
            
    print("\nScraping page:",page_url," = ", response)
    
    soup = bs(response.content, 'html.parser')
    
    return soup
    


In [149]:
pages_to_scrape = 4

current_page = 1

wine_df = pd.DataFrame(columns=['wine_name','link','country','type','description','evaluation','rating_count','lowest_price','full_price','discount'])

while (current_page != pages_to_scrape):
    
    wine_soup = get_soup(URL_BASE.format(current_page))
    
    if current_page == 1:
        pages_to_scrape = get_num_pages(wine_soup)
    
    new_df = scrape_soup(wine_soup)
    wine_df= wine_df.append(new_df)
    
    ###print(current_page)
    
    time.sleep(10)
    current_page += 1
    
print("Scrape finished.")
wine_df




Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p1.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p2.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p3.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p4.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p5.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p6.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p7.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p8.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO-p9.html  =  <Response [200]>

Scraping page: https://www.wine.com.br/vinhos/tinto/cVINHOS-atTIPO_TINTO

,wine_name,link,country,type,description,evaluation,rating_count,lowest_price,full_price,discount
0,Pérez Cruz Limited Edition D.O. Valle del Maip...,https://wine.com.br/vinhos/perez-cruz-limited-...,Chile,tinto,"Frutado, taninos presentes, médio corpo, fresc...",4.0,7,145.76,180.9,0.194251
1,Finca La Chamiza Martín Alsina Malbec 2014,https://wine.com.br/vinhos/finca-la-chamiza-ma...,Argentina,tinto,"Frutado, amadeirado, corpo de médio para encor...",0.0,0,292.82,495.9,0.409518
2,Domaine L'Ostal Cazes Grand Vin 2015,https://wine.com.br/vinhos/domaine-l-ostal-caz...,França,tinto,"Encorpado, boa acidez, taninos macios",0.0,0,258.71,335.9,0.229801
3,Maycas del Limarí Reserva Especial Pinot Noir ...,https://wine.com.br/vinhos/maycas-del-limari-r...,Chile,tinto,"Bom corpo, acidez presente, taninos sedosos, f...",4.5,23,166.94,206.9,0.193137
4,Pauillac De Lynch-Bages 2015,https://wine.com.br/vinhos/pauillac-de-lynch-b...,França,tinto,"Médio corpo, boa acidez, taninos presentes, fi...",3.0,1,364.59,630.9,0.422111
...,...,...,...,...,...,...,...,...,...,...
1,Norton Reserva Syrah 2014,https://wine.com.br/vinhos/norton-reserva-syra...,Argentina,tinto,"Na boca é aveludado e denso, com taninos deli...",0.0,0,135.18,159.9,0.154597
2,Columbia Crest Grand Estates Cabernet Sauvigno...,https://wine.com.br/vinhos/columbia-crest-gran...,Estados Unidos,tinto,Final persistente com notas de chocolate amargo.,0.0,0,152.82,179.9,0.150528
3,Motto Unabashed Zinfandel 2015,https://wine.com.br/vinhos/motto-unabashed-zin...,Estados Unidos,tinto,O paladar revela sabores de framboesas com cob...,0.0,0,152.82,179.9,0.150528
4,Antinori Santa Cristina I.G.T. Toscana Rosso 2017,https://wine.com.br/vinhos/antinori-santa-cris...,Itália,tinto,"Na boca apresenta boa estrutura, maciez e fres...",0.0,0,117.53,139.9,0.159900


In [150]:
wine_df.to_csv("winelist.csv",sep=';')